In [59]:
# *****************************************************************************
# *****************************************************************************
# *****************************************************************************
#   scrapeall.py
#
#   Author: Fritz Knack
#
#   Scrape every page at phrits.com as it exists on 2021-12-24.
#
#   General Comments
# *****************************
# See DESIGN_scrapeall.md for full worklog.
#
# 2021-12-24
# Meta and Initialization.
#
# *****************************************************************************
# *****************************************************************************
# *****************************************************************************

# *****************************
"""
Imports
*******************************

from webdriver_manager.chrome import ChromeDriverManager
Runs the Chrome engine

from splinter import Browser
Browser automation. Execute scripts, move the mouse, modify cookies,
fill forms, search and navigate by element, back, forward, etc.

from bs4 import BeautifulSoup as soup
HTML content parser.

import re
Regular expressions.

import sys
System calls (e.g., sys.exit())
"""
from webdriver_manager.chrome import ChromeDriverManager
from splinter import Browser
from bs4 import BeautifulSoup as bs
import re
import sys


# import pandas as pd
# import urllib.request
import requests
# *****************************************************************************


# *****************************************************************************
# def error_out
"""
def error_out(e, msg_exit = ''):
Just a wrapper for the try-except failures
"""
# *****************************
def error_out(e, msg_exit = ''):
    msg = 'Program Fail\n' \
        + '------------\n' \
        + msg_exit
    print(f'\n\n{e.__class__.__name__}')
    sys.exit(msg)

# *****************************************************************************
# def keep_internal_markup
"""
"""
# *****************************
# def keep_internal_markup(elem = ''):
#     r1 = re.compile("^(<.+>)")
#     r1 = re.compile(".*(<\/.+>)$")
#     out_string = re.sub(r1, '', elem)
#     out_string = re.sub(r2, '', out_string)
#     return out_string

# *****************************************************************************
# Assorted options. Might as well gather them early.
# *****************************
opt_browser = 'chrome' # splinter defaults to firefox
opt_headless = True
opt_url = 'https://phrits.com/content/parts/part_fat_tom.php'
opt_parser = 'html.parser'

"""
Init
*******************************
path_exe:           ChromeDriverManager requirement
browser:            application
session_reponse:    application sesssion (e.g., chrome browser
soup:               base soup object, the (sub)page in HTML


"""

# get the browser engine
try:
    path_exe = {'executable_path': ChromeDriverManager().install()}
except Exception as e:
    error_out(e,
        'Unable to launch find path for ChromeDriverManager.install().'
        )

# get the browser session
try:
    browser = Browser(opt_browser, **path_exe, headless=opt_headless)
    print(f'browser: {browser}')
except Exception as e:
    error_out(e,
        f'Unable to create a session for URL {opt_url}'
        )

# go to the page
try:
    browser.visit(opt_url)
    session_reponse = browser.html
except Exception as e:
    error_out(e,
        f'Unable to retrieve page for URL {opt_url}'
        )

# Waiting in a hot tureen!
try: 
    soup = bs(session_reponse, opt_parser)
except Exception as e:
    error_out(e,
        f'Neither rich, nor green.\n' +
        'Couldn\'t make soup from URL {opt_url}'
        )




====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [C:\Users\micro\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache


browser: <splinter.driver.webdriver.chrome.WebDriver object at 0x000001F3DC7C65E0>


In [62]:
page = {
    "title": '',
    "h1_para": [],
    "h2s": [{
        "title": '',
        "h2_paras": []
    }]
}
outer_bands = soup.find_all("div", class_="outerband")

# for outer_band in outer_bands:
#     inner_bands = outer_band.find_all("div", class_="innerband")
#     for inner_band in inner_bands:
#         h2 = ''
#         h1s = inner_band.find_all("h1")
#         for h1 in h1s:

outer_band = outer_bands[0]
inner_bands = outer_band.find_all("div", class_="innerband")
inner_band = inner_bands[0]
h2 = ''
h1s = inner_band.find_all("h1")
h1 = h1s[0]
h2_counter = -1
page['title'] = h1.text
elem=h1.find_next()
while(elem):
    if elem.name == "h1":
        elem = None
    elif elem.name == 'p' and not h2:
        page["h1_para"].append(elem.contents)
    elif elem.name == 'h2':
        h2_counter += 1
        p_counter = 0
        h2 = elem.text
        h2_record = {
            "title": h2,
            "h2_paras": []
        }
        if h2_counter == 0:
            page['h2s'][h2_counter] = h2_record
        else:
            page['h2s'].append(h2_record)
    elif elem.name == 'p':
        if p_counter == 0:
            page['h2s'][h2_counter]['h2_paras'] = elem.contents
            p_counter += 1
        else:
            page['h2s'][h2_counter]['h2_paras'].append(elem.contents)
    elem = elem.find_next()
print(page)


{'title': 'Your Friend in the Kitchen: FAT TOM', 'h1_para': [['In a restaurant kitchen, or your own, food ', <i>should</i>, ' be healthful. But it ', <strong>must</strong>, ' be safe. Food safety includes protection from physical hazards: Think fish pin bones or tiny shards of metal from a can lid, for example. It also includes concern for chemical hazards: Cleaning supplies don’t belong in the pantry, and we don’t cook in galvanized—zinc-plated—pots and pans. But the concern that gets most of our attention is biological, the things that cause ', <a href="http://en.wikipedia.org/wiki/Foodborne_illness">foodborne illnesses</a>, ' or "food poisoning".'], ['We worry about viruses such as ', <a href="http://www.foodborneillness.com/hepatitis_food_poisoning/">Hepatitis A</a>, ' being transmitted from food handlers to eaters, but dangerous bacteria—pathogens—are the real bad guys, bringing all sorts of fun games to the party: ', <a href="http://www.cruisecritic.com/articles.cfm?ID=71">Norovi